In [1]:
from pprint import pprint

In [13]:
test = """
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
""".strip()

In [21]:
with open('input.txt', 'r') as f:
    input_ = f.read()

# Part 1

In [139]:
def parse_tree(lines):
    tree = current = {}
    dir_stack = [current]
    for line in lines:
        content = line.split()
        match content:
            case ['$', 'cd', '..']:
                dir_stack.pop()
                current = dir_stack[-1]
            
            case ['$', 'cd', where]:
                current[where] = {}
                current = current[where]
                dir_stack.append(current)
                
            case ['$', 'ls']:
                pass
            
            case ['dir', name]:
                pass

            case [size, name]:
                current[name] = size

    return tree


In [259]:
pprint(parse_tree(test.splitlines()))

{'/': {'a': {'e': {'i': '584'}, 'f': '29116', 'g': '2557', 'h.lst': '62596'},
       'b.txt': '14848514',
       'c.dat': '8504156',
       'd': {'d.ext': '5626152',
             'd.log': '8033020',
             'j': '4060174',
             'k': '7214296'}}}


In [260]:
def collect_sizes(tree, collect_dirs=0):
    sizes = {}
    tot = 0
    for key, value in tree.items():
        if type(value) is dict:
            #print('entering', key, sizes)
            sizes_dir, collect_dirs, tot_dir = collect_sizes(value, collect_dirs)
            if tot_dir <= 100_000:
                collect_dirs += tot_dir
            sizes[key] = sizes_dir, tot_dir
        else:
            sizes[key] = None, int(value)
        tot += sizes[key][1]
        #print('exiting', key, 'tot', tot, sizes)
    
    return sizes, collect_dirs, tot

In [261]:
sizes, collect_dirs, tot = collect_sizes(parse_tree(test.splitlines()))
collect_dirs

95437

In [262]:
sizes, collect_dirs, tot = collect_sizes(parse_tree(input_.splitlines()))
collect_dirs

1118405

# Part 2

In [263]:
def collect_dirs_to_delete(tree, collect_dirs=None):
    if collect_dirs is None:
        collect_dirs = {}
    sizes = {}
    tot = 0
    for key, value in tree.items():
        if type(value) is dict:
            #print('entering', key, sizes)
            sizes_dir, collect_dirs, tot_dir = collect_dirs_to_delete(value, collect_dirs)
            collect_dirs[key] = tot_dir
            sizes[key] = sizes_dir, tot_dir
        else:
            sizes[key] = None, int(value)
        tot += sizes[key][1]
        #print('exiting', key, 'tot', tot, sizes)
    
    return sizes, collect_dirs, tot

In [265]:
sizes, collect_dirs, tot = collect_dirs_to_delete(parse_tree(test.splitlines()))

In [266]:
need_free = 30000000 - (70000000 - tot)
need_free

8381165

In [267]:
eligible = {k: v for k,v in collect_dirs.items() if v > need_free}
min(eligible, key=lambda k: eligible[k])

'd'

In [268]:
sizes, collect_dirs, tot = collect_dirs_to_delete(parse_tree(input_.splitlines()))

In [269]:
need_free = 30000000 - (70000000 - tot)
need_free

10216456

In [270]:
eligible = {k: v for k,v in collect_dirs.items() if v > need_free}
min(eligible, key=lambda k: eligible[k])

'blvcjl'

In [271]:
eligible['blvcjl']

12545514